In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
test = pd.read_csv('ZIP_CBSA_122021.csv', nrows = 100)
test['zip'] = test['zip'].str.slice(start = 1, stop = 6)

test.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state
0,00683,41900,SAN GERMAN,PR
1,00683,32420,SAN GERMAN,PR
2,00923,41980,SAN JUAN,PR
3,01010,44140,BRIMFIELD,MA
4,01010,49340,BRIMFIELD,MA


In [4]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

for chunk in tqdm(pd.read_csv('nppes_with_taxonomy.csv', dtype = {'postal_code':str}, chunksize = 1000)):
    chunk['postal_code'] = chunk['postal_code'].str.slice(stop=5)
    chunk.to_sql('nppes', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('nucc_taxonomy_220.csv', chunksize = 1000)):
    chunk.to_sql('taxonomy', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('ZIP_CBSA_122021.csv', chunksize = 1000, )):
    chunk['zip'] = chunk['zip'].str.slice(start = 1, stop = 6)
    chunk.to_sql('cbsa', hop_proj, if_exists = 'append', index = False)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

PermissionError: [Errno 13] Permission denied

In [5]:
for chunk in tqdm(pd.read_csv('hoptrim.csv', 
                              dtype = {'zip':str},
                              chunksize = 1000)):
        chunk.to_sql('hop', hop_proj, if_exists = 'append', index = False)

0it [00:00, ?it/s]

In [6]:
hop_proj.close()

In [8]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

hop_proj.execute('CREATE INDEX npi ON nppes(npi)')
hop_proj.execute('CREATE INDEX postal_code ON nppes(postal_code)')
hop_proj.execute('CREATE INDEX primary_taxonomy ON nppes(primary_taxonomy)')

hop_proj.execute('CREATE INDEX from_npi ON hop(from_npi)')
hop_proj.execute('CREATE INDEX to_npi ON hop(to_npi)')

hop_proj.execute('CREATE INDEX zip ON cbsa(zip)')

hop_proj.execute('CREATE INDEX code ON taxonomy(code)')

hop_proj.close()

In [24]:
db = sqlite3.connect('hop_proj.sqlite')

query = "SELECT * FROM taxonomy LIMIT 1"

tax_sqlite = pd.read_sql(query, db)

db.close()

In [25]:
tax_sqlite

,code,grouping,classification,specialization,class_spec
0,193200000X,Group,Multi-Specialty,None,Multi-Specialty


In [26]:
db = sqlite3.connect('hop_proj.sqlite')

query = "SELECT * FROM nppes LIMIT 1"

nppes_sqlite = pd.read_sql(query, db)

db.close()

In [27]:
nppes_sqlite

,npi,entity_code,org_name,last_name,first_name,middle,prefix,suffix,credential,address1,address2,city,state,postal_code,primary_taxonomy
0,1679576722,1,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,68847,207X00000X


In [ ]:
db = sqlite3.connect('hop_proj.sqlite')

query = "SELECT * FROM nppes LIMIT 1"

nppes_sqlite = pd.read_sql(query, db)

db.close()